# Pandas Notes

Sample data is installed from kaggle: https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance

In [27]:
import pandas as pd

## import csv file
df_full = pd.read_csv("student_habits_performance.csv")
print(df_full.columns)

Index(['student_id', 'age', 'gender', 'study_hours_per_day',
       'social_media_hours', 'netflix_hours', 'part_time_job',
       'attendance_percentage', 'sleep_hours', 'diet_quality',
       'exercise_frequency', 'parental_education_level', 'internet_quality',
       'mental_health_rating', 'extracurricular_participation', 'exam_score'],
      dtype='object')


In [28]:
## Select specific columns of dataframe
df = df_full[["student_id", "age", "gender","study_hours_per_day","social_media_hours"]]
print(df.head())

  student_id  age  gender  study_hours_per_day  social_media_hours
0      S1000   23  Female                  0.0                 1.2
1      S1001   20  Female                  6.9                 2.8
2      S1002   21    Male                  1.4                 3.1
3      S1003   23  Female                  1.0                 3.9
4      S1004   19  Female                  5.0                 4.4


In [29]:
## print basic information of dataframe

print(df.columns, "\n")
print(df.shape)

Index(['student_id', 'age', 'gender', 'study_hours_per_day',
       'social_media_hours'],
      dtype='object') 

(1000, 5)


In [30]:
## Conditional subset 

female_df_high = df[(df["gender"]=="Female") & (df["social_media_hours"]>=5)]
print(female_df_high.head())

    student_id  age  gender  study_hours_per_day  social_media_hours
58       S1058   19  Female                  2.2                 5.0
145      S1145   23  Female                  2.2                 6.2
311      S1311   23  Female                  2.6                 5.3
368      S1368   19  Female                  2.8                 5.0
384      S1384   21  Female                  2.6                 5.4


In [32]:
## Sort by age then social_media_hours 

female_df_high = female_df_high.sort_values(["age", "social_media_hours"], ascending=[True, False])
print(female_df_high)

    student_id  age  gender  study_hours_per_day  social_media_hours
678      S1678   18  Female                  3.5                 5.2
414      S1414   19  Female                  4.2                 5.6
58       S1058   19  Female                  2.2                 5.0
368      S1368   19  Female                  2.8                 5.0
384      S1384   21  Female                  2.6                 5.4
145      S1145   23  Female                  2.2                 6.2
311      S1311   23  Female                  2.6                 5.3
829      S1829   24  Female                  4.1                 5.4


In [ ]:
## Count values of 'exercise_frequency'

print(df_full['exercise_frequency'].value_counts().sort_index())

exercise_frequency
0    144
1    146
2    122
3    153
4    134
5    149
6    152
Name: count, dtype: int64
